In [2]:
import pandas as pd
import os

In [3]:
interim_dir = "../data/interim"
processed_dir = "../data/processed"
data_model_file = os.path.join(processed_dir, "data_model_output.csv")

In [4]:
data_model_df = pd.read_csv(data_model_file)

C:\Users\USVV724227\AppData\Local\Temp\ipykernel_34116\3792778259.py:1: DtypeWarning: Columns (1,13,14,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,47,48,54,55,59,65,69,74,77,78,80,82,83,86,93,95,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,132,142,147,150,151,152,155,156,157,167,169,177,179,196,197,198,201,210,211,216,223,237,239,240,241,242,243,244,246,248,249,273,283,284,285,288,291,295,296,297,299,300,335,337,338,340,411,414) have mixed types. Specify dtype option on import or set low_memory=False.
  data_model_df = pd.read_csv(data_model_file)


In [5]:
working_df = data_model_df[(data_model_df['is_completed'] == 1) & (data_model_df['record_type_synthetic'] == 0) & (data_model_df['initial_etc_check']==True) & (data_model_df['validation_severity_person']!='Critical') & (data_model_df['validation_severity_trip'] != 'Critical')]

In [6]:
# Create raw count crosstab
count_ct = pd.crosstab(working_df['thanksgiving_week_flag'], working_df['resident_visitor_purpose_label'])

# Add 'Total' column (row-wise sum)
count_ct['Total'] = count_ct.sum(axis=1)

# Create percentage crosstab (row-wise)
percent_ct = pd.crosstab(working_df['thanksgiving_week_flag'], working_df['resident_visitor_purpose_label'], normalize='index') * 100

# Add a 'Total' column to percentages (always 100%)
percent_ct['Total'] = 100.0

# Format percentage to 1 decimal place and add %
percent_ct = percent_ct.round(1).astype(str) + '%'

# Combine counts and percentages
combined_ct = count_ct.astype(str) + ' (' + percent_ct + ')'

# Optional: Rename index/column names for clarity
combined_ct.index.name = 'Thanksgiving Week'
combined_ct.columns.name = 'Purpose'


In [7]:
combined_ct

Purpose,EMPLOYEE,RESIDENT_BUSINESS,RESIDENT_NON_BUSINESS,VISITOR_BUSINESS,VISITOR_NON_BUSINESS,Total
Thanksgiving Week,,,,,,
False,659 (13.2%),456 (9.2%),1310 (26.3%),868 (17.4%),1682 (33.8%),4975 (100.0%)
True,0 (0.0%),5 (9.4%),27 (50.9%),4 (7.5%),17 (32.1%),53 (100.0%)


In [8]:
# Ensure date_completed is datetime
working_df['date_completed'] = pd.to_datetime(working_df['date_completed'])

# Define the Thanksgiving week Wednesday to Friday range
thanksgiving_start = pd.Timestamp('2024-11-27')  # Wednesday
thanksgiving_end = pd.Timestamp('2024-11-29')    # Friday

# Flag for Thanksgiving Wed-Fri
working_df['thanksgiving_wed_fri_flag'] = working_df['date_completed'].between(thanksgiving_start, thanksgiving_end)

# Flag for Wed-Fri of all other weeks (not Thanksgiving Wed-Fri)
working_df['other_wed_fri_flag'] = (
    (working_df['date_completed'].dt.weekday >= 2) &  # Wednesday=2, Thursday=3, Friday=4
    (working_df['date_completed'].dt.weekday <= 4) &
    (~working_df['thanksgiving_wed_fri_flag'])
)

# Filter data for only these two groups
filtered_df = working_df[working_df['thanksgiving_wed_fri_flag'] | working_df['other_wed_fri_flag']].copy()

# Create a flag column for clarity
filtered_df['wed_fri_period'] = filtered_df['thanksgiving_wed_fri_flag'].map(
    {True: 'Thanksgiving Wed-Fri', False: 'Other Weeks Wed-Fri'}
)

# Create raw count crosstab
count_ct = pd.crosstab(filtered_df['wed_fri_period'], filtered_df['resident_visitor_purpose_label'])

# Add 'Total' column (row-wise sum)
count_ct['Total'] = count_ct.sum(axis=1)

# Create percentage crosstab (row-wise)
percent_ct = pd.crosstab(filtered_df['wed_fri_period'], filtered_df['resident_visitor_purpose_label'], normalize='index') * 100

# Add a 'Total' column to percentages (always 100%)
percent_ct['Total'] = 100.0

# Format percentage to 1 decimal place and add %
percent_ct = percent_ct.round(1).astype(str) + '%'

# Combine counts and percentages
combined_ct = count_ct.astype(str) + ' (' + percent_ct + ')'

# Optional: Rename index/column names for clarity
combined_ct.index.name = 'Wed-Fri Period'
combined_ct.columns.name = 'Purpose'

# Display the result
combined_ct


C:\Users\USVV724227\AppData\Local\Temp\ipykernel_34116\2855038743.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['date_completed'] = pd.to_datetime(working_df['date_completed'])
C:\Users\USVV724227\AppData\Local\Temp\ipykernel_34116\2855038743.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['thanksgiving_wed_fri_flag'] = working_df['date_completed'].between(thanksgiving_start, thanksgiving_end)
C:\Users\USVV724227\AppData\Local\Temp\ipykernel_34116\2855038743.py:12: SettingWit

Purpose,EMPLOYEE,RESIDENT_BUSINESS,RESIDENT_NON_BUSINESS,VISITOR_BUSINESS,VISITOR_NON_BUSINESS,Total
Wed-Fri Period,,,,,,
Other Weeks Wed-Fri,429 (16.7%),182 (7.1%),815 (31.8%),559 (21.8%),580 (22.6%),2565 (100.0%)
Thanksgiving Wed-Fri,0 (0.0%),0 (0.0%),9 (75.0%),0 (0.0%),3 (25.0%),12 (100.0%)


In [9]:
# Ensure date_completed is datetime
working_df['date_completed'] = pd.to_datetime(working_df['date_completed'])

# Thanksgiving week date ranges
tg_mon_start = pd.Timestamp('2024-11-25')  # Monday
tg_tue_end = pd.Timestamp('2024-11-26')    # Tuesday
tg_wed_start = pd.Timestamp('2024-11-27')  # Wednesday
tg_thu_end = pd.Timestamp('2024-11-28')    # Thursday
tg_fri_start = pd.Timestamp('2024-11-29')  # Friday
tg_sun_end = pd.Timestamp('2024-12-01')    # Sunday

# Flags for Thanksgiving periods
working_df['tg_mon_tue_flag'] = working_df['date_completed'].between(tg_mon_start, tg_tue_end)
working_df['tg_wed_thu_flag'] = working_df['date_completed'].between(tg_wed_start, tg_thu_end)
working_df['tg_fri_sun_flag'] = working_df['date_completed'].between(tg_fri_start, tg_sun_end)

# Flags for non-Thanksgiving periods
# Monday (0) or Tuesday (1), but not Thanksgiving week
working_df['other_mon_tue_flag'] = (
    (working_df['date_completed'].dt.weekday.isin([0, 1])) &
    (~working_df['tg_mon_tue_flag'])
)
# Wednesday (2) or Thursday (3), but not Thanksgiving week
working_df['other_wed_thu_flag'] = (
    (working_df['date_completed'].dt.weekday.isin([2, 3])) &
    (~working_df['tg_wed_thu_flag'])
)
# Friday (4), Saturday (5), Sunday (6), but not Thanksgiving week
working_df['other_fri_sun_flag'] = (
    (working_df['date_completed'].dt.weekday.isin([4, 5, 6])) &
    (~working_df['tg_fri_sun_flag'])
)

# Combine into a new period label
def period_label(row):
    if row['tg_mon_tue_flag']:
        return 'Thanksgiving Mon-Tue'
    elif row['tg_wed_thu_flag']:
        return 'Thanksgiving Wed-Thu'
    elif row['tg_fri_sun_flag']:
        return 'Thanksgiving Fri-Sun'
    elif row['other_mon_tue_flag']:
        return 'Other Weeks Mon-Tue'
    elif row['other_wed_thu_flag']:
        return 'Other Weeks Wed-Thu'
    elif row['other_fri_sun_flag']:
        return 'Other Weeks Fri-Sun'
    else:
        return None

working_df['period_label'] = working_df.apply(period_label, axis=1)

# Filter to only relevant periods
filtered_df = working_df[working_df['period_label'].notnull()].copy()

# Create raw count crosstab
count_ct = pd.crosstab(filtered_df['period_label'], filtered_df['resident_visitor_purpose_label'])
count_ct['Total'] = count_ct.sum(axis=1)

# Create percentage crosstab (row-wise)
percent_ct = pd.crosstab(filtered_df['period_label'], filtered_df['resident_visitor_purpose_label'], normalize='index') * 100
percent_ct['Total'] = 100.0
percent_ct = percent_ct.round(1).astype(str) + '%'

# Combine counts and percentages
combined_ct = count_ct.astype(str) + ' (' + percent_ct + ')'

# Optional: Rename index/column names for clarity
combined_ct.index.name = 'Period'
combined_ct.columns.name = 'Purpose'

# Display the result
combined_ct


C:\Users\USVV724227\AppData\Local\Temp\ipykernel_34116\2076532943.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['date_completed'] = pd.to_datetime(working_df['date_completed'])
C:\Users\USVV724227\AppData\Local\Temp\ipykernel_34116\2076532943.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['tg_mon_tue_flag'] = working_df['date_completed'].between(tg_mon_start, tg_tue_end)
C:\Users\USVV724227\AppData\Local\Temp\ipykernel_34116\2076532943.py:14: SettingWithCopyWarning: 
A valu

Purpose,EMPLOYEE,RESIDENT_BUSINESS,RESIDENT_NON_BUSINESS,VISITOR_BUSINESS,VISITOR_NON_BUSINESS,Total
Period,,,,,,
Other Weeks Fri-Sun,118 (8.3%),106 (7.4%),391 (27.4%),286 (20.0%),527 (36.9%),1428 (100.0%)
Other Weeks Mon-Tue,173 (10.5%),213 (13.0%),341 (20.8%),182 (11.1%),733 (44.6%),1642 (100.0%)
Other Weeks Wed-Thu,368 (19.3%),137 (7.2%),578 (30.3%),400 (21.0%),422 (22.2%),1905 (100.0%)
Thanksgiving Fri-Sun,0 (0.0%),5 (20.0%),7 (28.0%),0 (0.0%),13 (52.0%),25 (100.0%)
Thanksgiving Mon-Tue,0 (0.0%),0 (0.0%),13 (65.0%),4 (20.0%),3 (15.0%),20 (100.0%)
Thanksgiving Wed-Thu,0 (0.0%),0 (0.0%),7 (87.5%),0 (0.0%),1 (12.5%),8 (100.0%)


In [10]:
# Define the desired order
period_order = [
    'Thanksgiving Mon-Tue',
    'Thanksgiving Wed-Thu',
    'Thanksgiving Fri-Sun',
    'Other Weeks Mon-Tue',
    'Other Weeks Wed-Thu',
    'Other Weeks Fri-Sun'
]

# Set period_label as a categorical with the desired order
filtered_df['period_label'] = pd.Categorical(
    filtered_df['period_label'],
    categories=period_order,
    ordered=True
)

# Create raw count crosstab
count_ct = pd.crosstab(filtered_df['period_label'], filtered_df['resident_visitor_purpose_label'])
count_ct['Total'] = count_ct.sum(axis=1)

# Create percentage crosstab (row-wise)
percent_ct = pd.crosstab(filtered_df['period_label'], filtered_df['resident_visitor_purpose_label'], normalize='index') * 100
percent_ct['Total'] = 100.0
percent_ct = percent_ct.round(1).astype(str) + '%'

# Combine counts and percentages
combined_ct = count_ct.astype(str) + ' (' + percent_ct + ')'

# Optional: Rename index/column names for clarity
combined_ct.index.name = 'Period'
combined_ct.columns.name = 'Purpose'

# Reindex to enforce desired order (in case crosstab drops empty categories)
combined_ct = combined_ct.reindex(period_order)

# Display the result
combined_ct


Purpose,EMPLOYEE,RESIDENT_BUSINESS,RESIDENT_NON_BUSINESS,VISITOR_BUSINESS,VISITOR_NON_BUSINESS,Total
Period,,,,,,
Thanksgiving Mon-Tue,0 (0.0%),0 (0.0%),13 (65.0%),4 (20.0%),3 (15.0%),20 (100.0%)
Thanksgiving Wed-Thu,0 (0.0%),0 (0.0%),7 (87.5%),0 (0.0%),1 (12.5%),8 (100.0%)
Thanksgiving Fri-Sun,0 (0.0%),5 (20.0%),7 (28.0%),0 (0.0%),13 (52.0%),25 (100.0%)
Other Weeks Mon-Tue,173 (10.5%),213 (13.0%),341 (20.8%),182 (11.1%),733 (44.6%),1642 (100.0%)
Other Weeks Wed-Thu,368 (19.3%),137 (7.2%),578 (30.3%),400 (21.0%),422 (22.2%),1905 (100.0%)
Other Weeks Fri-Sun,118 (8.3%),106 (7.4%),391 (27.4%),286 (20.0%),527 (36.9%),1428 (100.0%)
